# **Coordinate to impulse space Neural Network** 

In this notebbok I will attempt to create Neural Network that learns transformation from coordinate to impulse space. 

Theoretically it should work well for particles that were born in origin.

Here is the main notebook where I train the network.

In [1]:
# Imports

import os

import numpy as np
import pandas as pd

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

# Import keras

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Lambda
from keras.initializers import TruncatedNormal

Using TensorFlow backend.


In [2]:
np.set_printoptions(precision=3, suppress = True)

In [3]:
# Load data

data_path = '../Data/generated_data/'
x_train_df = pd.read_csv(os.path.join(data_path, "Coordinates_to_impulse_X.csv"))
y_train_df = pd.read_csv(os.path.join(data_path, "Coordinates_to_impulse_Y.csv"))
x_train = np.transpose(np.asarray([x_train_df["0"], x_train_df["1"], x_train_df["2"]]))
y_train = np.transpose(np.asarray([y_train_df["0"], y_train_df["1"], y_train_df["2"]]))

In [4]:
max_coordinates = np.amax(x_train, axis=0)
print(max_coordinates)

[1024.6  1024.73 2955.5 ]


In [5]:
max_impulses = np.amax(y_train, axis=0)
print(max_impulses)

[ 16.695  11.169 150.701]


Enhance data
---

In [6]:
#ganerate polynomial features
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3,  include_bias=False)
x_train_poly = poly.fit_transform(x_train)

In [7]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler().fit(x_train_poly)
x_train_poly = x_scaler.transform(x_train_poly) 

y_scaler = StandardScaler().fit(y_train)
y_train_scaled = y_scaler.transform(y_train)  

In [8]:
print(x_train[1])
print(x_train_poly[1])

[ -553.203    39.266 -1215.5  ]
[-2.27   0.152 -1.122  1.743 -0.235  1.823 -0.415 -0.129  0.18  -1.425
  0.225 -1.577 -0.009  0.206 -0.963 -0.012  0.001  0.076 -0.316]


In [9]:
print(y_train[1])
print(y_train_scaled[1])

[-0.144  0.177 -0.429]
[-0.173  0.239 -0.043]


Model
---

In [ ]:
# Define model
def model_architecture():
    model = Sequential()
    
    #initializer = TruncatedNormal(stddev=0.1)
    
    # Normalization layer
    # model.add(Lambda(lambda x: x / 1000))
    
    model.add(Dense(10000, activation='tanh'))
    
    model.add(Dense(1))    
    
    return model

In [ ]:
def train_model(model, X, Y, learning_rate = 0.001, batch_size=1000, epochs=100):   
    # Compiling the model
    optimizer = Adam(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    
    # Running and evaluating the model
    history = model.fit(X, Y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1)

Test model
---

In [ ]:
# Init model
model_check = model_architecture()

#sanity check data
y_check = np.linalg.norm(x_train, axis=1)
#y_check = np.sum(x_train, axis=1)

#model sanity check
train_model(model_check, x_train_poly, y_check)

In [ ]:
predict_check = model_check.predict(x_train_poly[0:5], batch_size=5)
print(predict_check[0:5])
print(y_check[0:5])

Train model
---

In [ ]:
#train model for px
model_px = train_model(x_train, y_train[:,0], epochs=100)

In [ ]:
#train model for px
model_py = train_model(x_train, y_train[:,1])

In [ ]:
#train model for px
model_pz = train_model(x_train, y_train[:,2])

In [ ]:
lim = 10
np.set_printoptions(precision=3, suppress = True)
predict_px = model_px.predict(x_train[0:lim], batch_size=lim)
predict_py = model_py.predict(x_train[0:lim], batch_size=lim)
predict_pz = model_pz.predict(x_train[0:lim], batch_size=lim)
predict = np.concatenate((predict_px, predict_py, predict_pz), axis=1)
print(predict[0:lim])
print(y_train[0:lim])
#print(x_train[0:lim])

Try clustering
---

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

def predict(X, eps=0.01):
    cl = DBSCAN(eps, min_samples=1, algorithm='kd_tree')
    labels = cl.fit_predict(StandardScaler().fit_transform(X))
    return labels

#score
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def try_clustering(hits, X):
    labels = predict(X)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

In [ ]:
data_path = '../Data/train_100_events/'
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join(data_path, event_prefix))

X = np.transpose([hits.x.values, hits.y.values, hits.z.values])

y_predict = model.predict(X, batch_size=10000)

try_clustering(hits, y_predict)